<a href="https://colab.research.google.com/github/rajni-arora/Deep-Learning-Projects/blob/main/ElasticSearch_haystack_on_Colab_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,preprocessing,elasticsearch,inference]

In [3]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [4]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [5]:
import time

time.sleep(30)

In [7]:
import requests
res = requests.get('http://localhost:9200/_cluster/health')

res.json()

{'cluster_name': 'elasticsearch',
 'status': 'yellow',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'active_primary_shards': 2,
 'active_shards': 2,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 2,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 50.0}

In [35]:
with open("/content/test_reader.txt", 'r') as file:
  data = file.read()

'Ins'

In [38]:
import requests
#data = requests.get('https://raw.githubusercontent.com/jamescalam/transformers/main/data/text/meditations/clean.txt')
text = data.split('\n')
text[:3]

['Instant payments',
 'Retail instant payment systems (instant payment systems) process small-value interbank',
 'transfers such that funds are available nearly instantly, as opposed to the potentially multiday settlement period for retail transfers on certain legacy bank payment systems.48 Instant']

In [39]:

data_json = [
    {
        'content': paragraph,
        'meta': {
            'source': 'meditations'
        }
    } for paragraph in text
]

In [42]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")

document_store = ElasticsearchDocumentStore(host=host, username="", password="", index="data")

In [40]:
data_json[:3]

[{'content': 'Instant payments', 'meta': {'source': 'meditations'}},
 {'content': 'Retail instant payment systems (instant payment systems) process small-value interbank',
  'meta': {'source': 'meditations'}},
 {'content': 'transfers such that funds are available nearly instantly, as opposed to the potentially multiday settlement period for retail transfers on certain legacy bank payment systems.48 Instant',
  'meta': {'source': 'meditations'}}]

In [43]:
document_store.write_documents(data_json)

In [45]:
requests.get('http://localhost:9200/data/_count').json()

{'count': 43,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [26]:
!pip install farm-haystack[inference]

In [30]:
#from haystack.retriever.sparse import ElasticsearchRetriever
from haystack.nodes import TfidfRetriever
from haystack.nodes import FARMReader
model_name_or_path = "deepset/roberta-base-squad2"
reader = FARMReader(model_name_or_path, use_gpu=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [46]:

retriever = TfidfRetriever(document_store)  # BM25
reader = FARMReader(model_name_or_path='deepset/bert-base-cased-squad2',
                    context_window_size=1500,
                    use_gpu=True)

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [47]:
from haystack.pipelines import ExtractiveQAPipeline

qa = ExtractiveQAPipeline(reader=reader, retriever=retriever)

In [49]:
richa = qa.run(query='What is instant payment?')

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  5.13 Batches/s]


In [50]:
richa.to_csv("result")

AttributeError: 'dict' object has no attribute 'to_csv'